In [1]:
%set_env CUDA_VISIBLE_DEVICES=3

env: CUDA_VISIBLE_DEVICES=3


In [2]:
import re
import os
import tensorflow as tf
import numpy as np
import torch
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from transformers import DistilBertTokenizerFast, TFBertModel, BertConfig
from alibi.explainers import IntegratedGradients
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence

In [3]:
def preprocess_reviews(reviews):
    
    REPLACE_NO_SPACE = re.compile("[.;:,!\'?\"()\[\]]")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

def process_sentences(sentence1, 
                      tokenizer, 
                      max_len, 
                      add_special_tokens=True):

    z = tokenizer(sentence1, 
                  add_special_tokens = add_special_tokens, 
                  padding = 'max_length', 
                  max_length = max_len, truncation = True,
                  return_token_type_ids=True, 
                  return_attention_mask = True,  
                  return_tensors = 'np')

    return [z['input_ids'], z['attention_mask']]

def decode_sentence(x, reverse_index):
    # the `-3` offset is due to the special tokens used by keras
    # see https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset
    return " ".join([reverse_index.get(i - 3, 'UNK') for i in x])

# Load and process data

Load imdb dataset

In [4]:
max_features = 10000
max_len = 100
add_special_tokens=False

In [5]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
test_labels = y_test.copy()
train_labels = y_train.copy()
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

index = imdb.get_word_index()
reverse_index = {value: key for (key, value) in index.items()} 

Loading data...


Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 100)
x_test shape: (25000, 100)


# Automodel

In this session, we will use the tensorflow auto model for sequence classification. This model is already finetuned for sentiment analisys and do not require training

In [6]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
auto_model_bert = TFAutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


The automodel output is a custom object containing the output logits. We use a wrapper to transform this output in a tensor and apply a softmax function on the logits.

In [7]:
class AutoModelWrapper(tf.keras.Model):

    def __init__(self, model_bert, **kwargs):
        super(AutoModelWrapper, self).__init__()
        self.model_bert = model_bert

    def call(self, inputs, attention_mask=None, output_nb=5):
        out = self.model_bert(inputs, attention_mask=attention_mask)
        return tf.nn.softmax(out.logits)
        #return out
    
    def get_config(self):
        return {}

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [8]:
frozenModelOut = AutoModelWrapper(auto_model_bert)

# Calculate ingrated gradients

Here we calculate the integrated gradients for the first 10 reviews in the test set

In [9]:
#x_test_sample = ["I love you, I like you",
#                "I love you, I like you, I also kinda dislike you",
#                "I hate you, I dislike you, but maybe not"]
x_test_sample = [decode_sentence(x_test[i], reverse_index) for i in range(10)]
x_test_sample = preprocess_reviews(x_test_sample)
x_test_sample = process_sentences(x_test_sample, 
                                   tokenizer, 
                                   max_len,
                                   add_special_tokens=add_special_tokens)
kwargs = {'attention_mask': x_test_sample[1]}

In [10]:
auto_out = frozenModelOut(x_test_sample[0], **kwargs)
auto_out

<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[9.9760783e-01, 2.3921041e-03],
       [5.7495857e-04, 9.9942505e-01],
       [3.7964460e-02, 9.6203548e-01],
       [9.9770999e-01, 2.2900314e-03],
       [3.7964175e-03, 9.9620360e-01],
       [9.9792743e-01, 2.0725005e-03],
       [9.0457928e-01, 9.5420703e-02],
       [9.9938536e-01, 6.1466231e-04],
       [9.2592180e-01, 7.4078158e-02],
       [1.7677458e-03, 9.9823231e-01]], dtype=float32)>

Choosing which to which embeddings block we want to calculate the attributions for. 

In [11]:
bl = frozenModelOut.layers[0].layers[0].transformer.layer[1]
bl

In [12]:
n_steps = 5
method = "gausslegendre"
internal_batch_size = 5
ig  = IntegratedGradients(frozenModelOut,
                          layer=bl,
                          n_steps=n_steps, 
                          method=method,
                          internal_batch_size=internal_batch_size)

In [13]:
predictions = frozenModelOut(x_test_sample[0], **kwargs).numpy().argmax(axis=1)
explanation = ig.explain(x_test_sample[0], 
                         forward_kwargs=kwargs,
                         baselines=None, 
                         target=predictions)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [14]:
# Get attributions values from the explanation object
attrs = explanation.attributions[0]
print('Attributions shape:', attrs.shape)

Attributions shape: (10, 100, 768)


In [15]:
attrs = attrs.sum(axis=2)
print('Attributions shape:', attrs.shape)

Attributions shape: (10, 100)


In [16]:
i = 1
x_i = x_test_sample[0][i]
attrs_i = attrs[i]
pred = predictions[i]
pred_dict = {1: 'Positive review', 0: 'Negative review'}

In [17]:
from IPython.display import HTML
def  hlstr(string, color='white'):
    """
    Return HTML markup highlighting text with the desired color.
    """
    return f"<mark style=background-color:{color}>{string} </mark>"

In [18]:
def colorize(attrs, cmap='PiYG'):
    """
    Compute hex colors based on the attributions for a single instance.
    Uses a diverging colorscale by default and normalizes and scales
    the colormap so that colors are consistent with the attributions.
    """
    import matplotlib as mpl
    cmap_bound = np.abs(attrs).max()
    norm = mpl.colors.Normalize(vmin=-cmap_bound, vmax=cmap_bound)
    cmap = mpl.cm.get_cmap(cmap)
    
    # now compute hex values of colors
    colors = list(map(lambda x: mpl.colors.rgb2hex(cmap(norm(x))), attrs))
    return colors

In [19]:
words = tokenizer.decode(x_i).split()
colors = colorize(attrs_i)

In [20]:
print('Predicted label =  {}: {}'.format(pred, pred_dict[pred]))

Predicted label =  1: Positive review


In [21]:
y_test[i]

array([0., 1.], dtype=float32)

In [22]:
HTML("".join(list(map(hlstr, words, colors))))

# Fine tune bert with custom model's head

In this session we use a pretrained bert model combined with a custom model's head. We fine tune the model's head for sentiment analysis on the imdb dataset. The bert model is used to produced embeddings, which are then used as input features for a simple feed forward network. 

In [59]:
def train_generator():
    for s1, s2, l in zip(X_train[0], X_train[1], y_train):
        yield {'input_ids': s1, 'attention_mask': s2}, l
        
def test_generator():
    for s1, s2, l in zip(X_test[0], X_test[1], y_test):
        yield {'input_ids': s1, 'attention_mask': s2}, l

def get_embeddings(generator, model, batch_size=50, stop_after_batch=2):
    dataset = tf.data.Dataset.from_generator(generator,
                                             output_types=({'input_ids': tf.int64,
                                                              'attention_mask': tf.int64}, 
                                                             tf.int64))
    dataset = dataset.batch(batch_size)
    embbedings = []

    i = 0
    for X_batch in dataset:
        batch_embeddings = model(X_batch[0])
        embbedings.append(batch_embeddings.last_hidden_state.numpy())
        
        i += 1
        # Stopping the extraction after 3 batches for code testing,
        # since extracting embeddings for the whole dataset is slow.
        if stop_after_batch is not None:
            if i > stop_after_batch: 
                break

    return np.concatenate(embbedings, axis=0)

In [60]:
tokenizer = DistilBertTokenizerFast.from_pretrained('bert-base-uncased')
config = BertConfig.from_pretrained("bert-base-uncased", 
                                    output_hidden_states=True)
modelBert = TFBertModel.from_pretrained("bert-base-uncased",
                                        config=config)

modelBert.trainable=False

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


## Extract embeddings for training

NOTE (to delete): Here we extract the embedding for training. Extracting the embedding for the whole dataset is very slow. For the sake of testing the code, here the extraction is stopped after 3 batches of 50 sentences. 


In [74]:
batch_size_emb = 50
stop_after_batch = 2

In [75]:
X_train, X_test = [], []
for i in range(len(x_train)):
    tr_sentence = decode_sentence(x_train[i], reverse_index)
    X_train.append(tr_sentence)
    te_sentence = decode_sentence(x_test[i], reverse_index)
    X_test.append(te_sentence)

In [76]:
X_train = preprocess_reviews(X_train)
X_train = process_sentences(X_train, 
                           tokenizer, 
                           max_len, 
                           add_special_tokens=add_special_tokens)
X_test = preprocess_reviews(X_test)
X_test = process_sentences(X_test,
                           tokenizer, 
                           max_len,
                           add_special_tokens=add_special_tokens)

In [77]:
train_embbedings = get_embeddings(train_generator, 
                                  modelBert, 
                                  batch_size=batch_size_emb, 
                                  stop_after_batch = stop_after_batch)
test_embbedings = get_embeddings(test_generator, 
                                 modelBert, 
                                 batch_size=batch_size_emb, 
                                 stop_after_batch = stop_after_batch)

In [78]:
train_embbedings.shape

(150, 100, 768)

In [79]:
test_embbedings.shape

(150, 100, 768)

## Train model

In [80]:
load_model = False
save_model = False

In [81]:
nb_filters=64
dropout_1=0.4
dropout_2=0.
hidden_dims=128
batch_size = 128
epochs = 3
skip_conv=False

In [82]:
class ModelOut(tf.keras.Model):

    def __init__(self, 
                 nb_filters=32,
                 dropout_1=0.2,
                 dropout_2=0.2, 
                 hidden_dims=32,
                skip_conv=False):
        super(ModelOut, self).__init__()
        
        self.nb_filters = nb_filters
        self.dropout_1 = dropout_1
        self.dropout_2 = dropout_2
        self.hidden_dims = hidden_dims
        self.skip_conv = skip_conv
        
        if not self.skip_conv:
            self.conv = tf.keras.layers.Conv1D(nb_filters, 
                                               kernel_size=3, 
                                               padding="valid", 
                                               strides=1)
            #self.dropoutl_1 = tf.keras.layers.Dropout(dropout_1)
            self.maxpool = tf.keras.layers.GlobalMaxPool1D()
        else:
            self.flat = tf.keras.layers.Flatten()
            
        self.dense_1 =  tf.keras.layers.Dense(hidden_dims, 
                                              activation='relu')
        #self.dropoutl_2 = tf.keras.layers.Dropout(dropout_2)
        self.dense_2 = tf.keras.layers.Dense(2, 
                                             activation='softmax')

    def call(self, inputs):
        if not self.skip_conv:
            x = self.conv(inputs)
            #x = self.dropoutl_1(x)
            x = self.maxpool(x)
        else:
            x = self.flat(inputs)
            
        x = self.dense_1(x)
        #x = self.dropoutl_2(x)
        x = self.dense_2(x)
        return x
    
    def get_config(self):
        return {"nb_filters": self.nb_filters,
                "dropout_1": self.dropout_1,
                "dropout_2": self.dropout_2, 
                "hidden_dims": self.hidden_dims,
               "skip_conv": self.skip_conv}

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [83]:
model_out = ModelOut(nb_filters=nb_filters,
                 dropout_1=dropout_1,
                 dropout_2=dropout_2, 
                 hidden_dims=hidden_dims,
                    skip_conv=skip_conv)

In [84]:
batch_size * (stop_after_batch + 1)

384

In [85]:
filepath = './model_imdb_transformers/'  # change to directory where model is downloaded

model_out.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

if not load_model:
    
    checkpoint_path = os.path.join(filepath, "training_2/cp-{epoch:04d}.ckpt")
    checkpoint_dir = os.path.dirname(checkpoint_path)

    # Create a callback that saves the model's weights every 5 epochs
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path, 
        verbose=1, 
        save_weights_only=True,
        save_freq=batch_size)

    model_out.fit(train_embbedings, y_train[:batch_size_emb * (stop_after_batch + 1)], 
                  validation_data=(test_embbedings, y_test[:batch_size_emb * (stop_after_batch + 1)]),
                  epochs=epochs, 
                  batch_size=batch_size,
                  callbacks=[cp_callback],
                  verbose=1)
else:
    #model_out.save(filepath)
    epoch = 6
    load_path = os.path.join(filepath, f"training_2/cp-{epoch:04d}.ckpt")
    model_out.load_weights(load_path)

Epoch 1/3
2/2 [==============================] - 1s 343ms/step - loss: 0.9544 - accuracy: 0.4333 - val_loss: 1.1047 - val_accuracy: 0.5133
Epoch 2/3
2/2 [==============================] - 0s 128ms/step - loss: 0.8274 - accuracy: 0.5867 - val_loss: 0.8022 - val_accuracy: 0.4867
Epoch 3/3
2/2 [==============================] - 0s 137ms/step - loss: 0.8234 - accuracy: 0.4267 - val_loss: 0.8248 - val_accuracy: 0.4867


## Combine bert and over head model

In [86]:
class frozenModelOut(tf.keras.Model):

    def __init__(self, model_bert, model_out, **kwargs):
        super(frozenModelOut, self).__init__()
        self.model_bert = model_bert
        self.model_out = model_out

    def call(self, inputs, attention_mask=None, output_nb=5):
        out = self.model_bert(inputs, attention_mask=attention_mask)
        out = self.model_out(out.last_hidden_state)
        #return tf.nn.softmax(out.logits)
        return out
    
    def get_config(self):
        return {}

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [87]:
frozenModelOut = frozenModelOut(modelBert, model_out)

# Calculate ingrated gradients

In [88]:
#x_test_sample = ["I love you, I like you",
#                "I love you, I like you, I also kinda dislike you",
#                "I hate you, I dislike you, but maybe not"]
x_test_sample = [decode_sentence(x_test[i], reverse_index) for i in range(10)]
x_test_sample = preprocess_reviews(x_test_sample)
x_test_sample = process_sentences(x_test_sample, 
                                   tokenizer, 
                                   max_len,
                                   add_special_tokens=add_special_tokens)
kwargs = {'attention_mask': x_test_sample[1]}

In [89]:
out = frozenModelOut(x_test_sample[0], **kwargs)
out

<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[0.18093975, 0.81906027],
       [0.20847648, 0.7915235 ],
       [0.20010987, 0.79989016],
       [0.28533247, 0.7146675 ],
       [0.23448864, 0.76551133],
       [0.26529413, 0.7347058 ],
       [0.1632935 , 0.8367065 ],
       [0.19886269, 0.8011373 ],
       [0.17282802, 0.8271719 ],
       [0.20253125, 0.7974687 ]], dtype=float32)>

In [90]:
bl = frozenModelOut.layers[0].bert.encoder.layer[0]
bl

In [91]:
n_steps = 5
method = "gausslegendre"
internal_batch_size = 5
ig  = IntegratedGradients(frozenModelOut,
                          layer=bl,
                          n_steps=n_steps, 
                          method=method,
                          internal_batch_size=internal_batch_size)

In [92]:
predictions = frozenModelOut(x_test_sample[0], **kwargs).numpy().argmax(axis=1)
explanation = ig.explain(x_test_sample[0], 
                         forward_kwargs=kwargs,
                         baselines=None, 
                         target=predictions)

In [93]:
# Get attributions values from the explanation object
attrs = explanation.attributions[0]
print('Attributions shape:', attrs.shape)

Attributions shape: (10, 100, 768)


In [94]:
attrs = attrs.sum(axis=2)
print('Attributions shape:', attrs.shape)

Attributions shape: (10, 100)


In [95]:
i = 1
x_i = x_test_sample[0][i]
attrs_i = attrs[i]
pred = predictions[i]
pred_dict = {1: 'Positive review', 0: 'Negative review'}

In [96]:
from IPython.display import HTML
def  hlstr(string, color='white'):
    """
    Return HTML markup highlighting text with the desired color.
    """
    return f"<mark style=background-color:{color}>{string} </mark>"

In [97]:
def colorize(attrs, cmap='PiYG'):
    """
    Compute hex colors based on the attributions for a single instance.
    Uses a diverging colorscale by default and normalizes and scales
    the colormap so that colors are consistent with the attributions.
    """
    import matplotlib as mpl
    cmap_bound = np.abs(attrs).max()
    norm = mpl.colors.Normalize(vmin=-cmap_bound, vmax=cmap_bound)
    cmap = mpl.cm.get_cmap(cmap)
    
    # now compute hex values of colors
    colors = list(map(lambda x: mpl.colors.rgb2hex(cmap(norm(x))), attrs))
    return colors

In [98]:
words = tokenizer.decode(x_i).split()
colors = colorize(attrs_i)

In [99]:
print('Predicted label =  {}: {}'.format(pred, pred_dict[pred]))

Predicted label =  1: Positive review


In [100]:
y_test[i]

array([0., 1.], dtype=float32)

In [101]:
HTML("".join(list(map(hlstr, words, colors))))